In [1]:
import pandas as pd
import yfinance as yf
import plotly.express as px

In [2]:
def filter_market_hours(df):
    """Filter a datetime-indexed DataFrame to US market hours."""
    # Ensure timezone is ET
    df = df.tz_convert("America/New_York")

    # Filter weekdays (Monday=0 … Friday=4)
    df = df[df.index.dayofweek < 5]

    # Filter market time window
    market_open = pd.to_datetime("09:30").time()
    market_close = pd.to_datetime("16:00").time()
    df = df.between_time(market_open, market_close)

    return df


In [3]:
tickers = ["GDX", "GDXJ"]

data = yf.download(tickers, period="1mo", interval="5m")

# Convert to ET, filter market hours
data = data.tz_convert("America/New_York")
data = filter_market_hours(data)

# Flatten multi-index columns
data.columns = ["_".join(col) for col in data.columns]

# Prepare plotting dataframe
df = pd.DataFrame({
    "Datetime": data.index,
    "GDX": data["Close_GDX"],
    "GDXJ": data["Close_GDXJ"]
})


/var/folders/5t/wl4gg59n2351fgw4h64rs9hc0000gp/T/ipykernel_23781/63358834.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period="1mo", interval="5m")
[*********************100%***********************]  2 of 2 completed


In [4]:
fig = px.line(
    df,
    x="Datetime",
    y=["GDX", "GDXJ"],
    title="GDX vs GDXJ — Intraday Prices (Market Hours Only)",
    labels={"value": "Price", "Datetime": "Time (ET)"}
)

fig.update_layout(
    height=600,
    width=1200,
    xaxis_rangeslider_visible=False  # optional
)

fig.show()
